In [41]:
from transformers import AutoTokenizer, AutoModelForTokenClassification,AutoModelForCausalLM
from transformers import pipeline
import torch
from spacy.training import Example
import spacy
from collections import Counter
from nltk.util import ngrams

In [1]:
# #Atrolamma too big
# tokenizer = AutoTokenizer.from_pretrained(
#     pretrained_model_name_or_path="universeTBD/astrollama"
# )
# model = AutoModelForCausalLM.from_pretrained(
#     pretrained_model_name_or_path="universeTBD/astrollama",
#     device_map="auto",
# )

In [2]:
# model_name = "allenai/scibert_scivocab_uncased"  # SciBERT model

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForTokenClassification.from_pretrained(model_name)

# ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

In [3]:
# # Save model and tokenizer locally
# model.save_pretrained("./local_scibert_scivocab_uncased_model")
# tokenizer.save_pretrained("./local_scibert_scivocab_uncased_model")

In [26]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# # 1. Load a general-purpose NER model (fine-tuned for NER tasks)
# model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"  # Pre-trained NER model

# # Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForTokenClassification.from_pretrained(model_name)

# # 2. Create a NER pipeline
# ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# # 3. Define a list of known astronomical locations
# astronomical_locations = [
#     "Andromeda Galaxy", "Milky Way", "Mars", "Venus", "Earth", "Hubble Space Telescope", "Crab Nebula",
#     "Saturn", "Jupiter", "Alpha Centauri", "Orion Nebula", "Pluto", "Neptune", "Sirius", "Betelgeuse", "Sun"
# ]

# # 4. Define the function to prioritize astronomical locations
# def prioritize_astronomical_locations(ner_results):
#     prioritized_entities = []
#     i = 0
#     while i < len(ner_results):
#         entity = ner_results[i]
#         if entity['entity'].startswith("B-"):
#             full_entity = entity['word']
#             i += 1
#             while i < len(ner_results) and ner_results[i]['entity'].startswith("I-"):
#                 full_entity += " " + ner_results[i]['word']
#                 i += 1
#             if any(location in full_entity for location in astronomical_locations):
#                 prioritized_entities.append({"word": full_entity, "priority": "HIGH"})
#             else:
#                 prioritized_entities.append({"word": full_entity, "priority": "LOW"})
#         else:
#             i += 1
#     return prioritized_entities

# # 5. Define the function to process the input list of tokens and apply NER
# def process_astronomical_text(tokens):
#     text = " ".join(tokens)
#     ner_results = ner_pipeline(text)
#     prioritized_entities = prioritize_astronomical_locations(ner_results)
#     return prioritized_entities

# # 6. Example Usage
# tokens = ["The", "Andromeda", "Galaxy", "is", "located", "next", "to", "the", "Milky", "Way", "."]
# prioritized_entities = process_astronomical_text(tokens)

# # Display the results
# for entity in prioritized_entities:
#     print(f"Entity: {entity['word']}, Priority: {entity['priority']}")


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [42]:
# Load the base model
model = spacy.load("en_core_web_sm")
ner = model.get_pipe("ner")
ner.add_label("LOC")  # Add the label if not already present

0

In [124]:
import random

class AstroNERDataset:
    def __init__(self):
        self.templates = [
            "{location} is a popular spot for photographing {astro_object}.",
            "Astrophotographers recommend {location} for capturing {astro_object}.",
            "{location} offers some of the clearest skies for observing {astro_object}.",
            "{astro_object} can often be seen clearly from {location}.",
            "The skies over {location} are ideal for photographing {astro_object}.",
            "Many photographers visit {location} to capture {astro_object}.",
            "The observatories in {location} provide excellent views of {astro_object}.",
            "{astro_object} enthusiasts often gather at {location}.",
            "Photographing {astro_object} from {location} is an unforgettable experience.",
        ]

        self.locations = [
            "Mauna Kea", "Atacama Desert", "Death Valley", "Mont Mégantic", "Pic du Midi",
            "Lake Tekapo", "Okavango Delta", "Himalayas", "Cerro Paranal", "Uluru",
            "Big Bend National Park", "Lick Observatory", "Lowell Observatory",
            "International Space Station", "VLT in Chile", "Meteora in Greece"
        ]
        self.locations = [location.lower() for location in self.locations]

        self.astro_objects = [
            "Milky Way", "Moon", "Andromeda Galaxy", "Stars", "Nebulae", 
            "Planets", "Auroras", "Comets", "Meteors", "Exoplanets",
            "Deep-sky Objects", "Satellites", "The Sun", "Earth from Space",
            "Jupiter and its Moons", "The Orion Nebula"
        ]
        self.astro_objects = [astro_object.lower() for astro_object in self.astro_objects]

    
    def generate_example(self):
        """Generates a single labeled training example."""
        template = random.choice(self.templates)
        location = random.choice(self.locations)
        astro_object = random.choice(self.astro_objects)
        sentence = template.format(location=location, astro_object=astro_object)
        # Define entities for the example
        location_start = sentence.index(location)
        location_end = location_start + len(location)
        return (sentence, {"entities": [(location_start, location_end, "LOC")]})
    
    def generate_dataset(self, size=500):
        """Generates a dataset with the specified number of examples."""
        dataset = [self.generate_example() for _ in range(size)]
        return dataset

# Create the dataset
astro_dataset = AstroNERDataset()
train_data = astro_dataset.generate_dataset(size=500)

# Preview a few examples
for example in train_data[:5]:
    print(example)


('The skies over uluru are ideal for photographing andromeda galaxy.', {'entities': [(15, 20, 'LOC')]})
('Photographing comets from lowell observatory is an unforgettable experience.', {'entities': [(26, 44, 'LOC')]})
('satellites can often be seen clearly from okavango delta.', {'entities': [(42, 56, 'LOC')]})
('the orion nebula can often be seen clearly from big bend national park.', {'entities': [(48, 70, 'LOC')]})
('comets enthusiasts often gather at big bend national park.', {'entities': [(35, 57, 'LOC')]})


In [125]:
def fine_tune(model,train_data):# Convert to spaCy's Example objects
    examples = [Example.from_dict(model.make_doc(text), ann) for text, ann in train_data]

    # Fine-tune the model
    optimizer = model.resume_training()
    for epoch in range(10):
        losses = {}
        model.update(examples, drop=0.5, losses=losses)
fine_tune(model,train_data)

In [126]:
# Save the fine-tuned model
model.to_disk("fine_tuned_astro_ner")

In [61]:
tokens = [
    "The", "Mauna", "Kea", "Observatory", "in", "Hawaii", "offers", "unparalleled", "views", "of", "the", "Milky", "Way", "due", "to", "its", "high", "altitude", "and", "clear", "skies", ".",
    "The", "Atacama", "Desert", "in", "Chile", "is", "one", "of", "the", "best", "places", "on", "Earth", "for", "stargazing", ",", "thanks", "to", "its", "dry", "climate", "and", "minimal", "light", "pollution", ".",
    "Photographers", "love", "capturing", "the", "Aurora", "Borealis", "from", "Tromsø", ",", "Norway", ",", "especially", "during", "the", "winter", "months", "when", "the", "lights", "are", "most", "vivid", ".",
    "The", "Namib", "Desert", "in", "Namibia", "provides", "a", "spectacular", "view", "of", "the", "Southern", "Hemisphere", "'s", "stars", ",", "with", "almost", "no", "artificial", "light", "interference", ".",
    "The", "Canary", "Islands", "are", "popular", "for", "astrophotography", ",", "especially", "at", "the", "Roque", "de", "los", "Muchachos", "Observatory", "on", "La", "Palma", ".",
    "Utah", "'s", "Bryce", "Canyon", "National", "Park", "is", "a", "favorite", "for", "night", "photography", ",", "offering", "some", "of", "the", "darkest", "skies", "in", "the", "United", "States", ".",
    "The", "Dark", "Sky", "Park", "in", "Cherry", "Springs", "State", "Park", ",", "Pennsylvania", ",", "is", "renowned", "for", "its", "pristine", "night", "skies", "and", "excellent", "stargazing", "opportunities", ".",
    "Mount", "Cook", "National", "Park", "in", "New", "Zealand", "is", "part", "of", "the", "Aoraki", "Mackenzie", "International", "Dark", "Sky", "Reserve", ",", "making", "it", "a", "perfect", "spot", "for", "astrophotography", "."
]


In [127]:
model_t = spacy.load("fine_tuned_astro_ner")

In [ ]:

def get_locations(model,tokens):
    doc =  model(" ".join(tokens))
    for ent in doc.ents:
        locations = [ent.text for ent in doc.ents if ent.label_ in [ "LOC"]]
        return locations
get_locations(model_t,tokens)

['The Mauna Kea Observatory',
 'the Milky Way',
 'The Atacama Desert',
 'Earth',
 'the Aurora Borealis',
 'The Namib Desert',
 "the Southern Hemisphere 's",
 'The Canary Islands',
 'La Palma',
 'Bryce Canyon National Park',
 'The Dark Sky Park',
 'Cherry Springs State Park',
 'Mount Cook National Park']

In [139]:
def process_file(file_path,model):
    """
    Read sentences from a file and process them using spaCy's NER.
    
    Parameters:
    - file_path (str): Path to the input text file.
    """
    i = 0
    recomended_locs = []
    with open(file_path, "r") as file:
        lines = file.readlines()
    
    for line in lines:
        sentence = line.strip()  # Remove leading/trailing whitespace
        if sentence:  # Skip empty lines
            i += 1
            
            # Tokenize the sentence into words
            tokens = sentence.split()  # Split the sentence into tokens
            # Extract locations
            loc = get_locations(model, tokens)
            if loc != None:
                recomended_locs.extend(loc)
            
        if i == 1000:
            break
    return recomended_locs

# Example Usage: Replace 'input.txt' with your text file
rec_loc_c = process_file("preprocessed_comments_sentences",model_t)

In [140]:
print(rec_loc_c)

['jupiter', 'momentary trail meteor', 'bolide faint fuzzy ball', 'aircraft smoothly', 'starlink launch something', 'tail rocket launch', 'rocket launch bunch', 'rocket debris', 'white lights in formation just hanging in sky sometimes blinking drone', 'south carolina', 'skysafari app', 'naked eye on a good night', 'the easiest nebula', 'check out', 'georgia but', 'cheers mate', 'soooo cool', 'science olympiad astronomy', 'the sun nso kitt', 'molecules absorb', 'the dark sections', 'the dark band', 'made wallpaper', 'photo wallpaper', 'go outside and', 'really smart', 'stars always appear as', 'check over on rtelescopes', 'meteor showers', 'heavens above', 'gideon van buitenen', 'quick question', 'in suburbia', 'photo examples', 'hi mate', 'planetary detail', 'bst starguiders', 'check out star', 'a krait phantom', 'picture taken', 'astronomy stuff', 'the moon', 'the moon', 'the moon', 'laramie wyoming', 'photo shop', 'sct', 'eq6rpro mount', 'very rough polar alignment', 'the moon', 'the 

In [141]:
def process_file(file_path, model):
    """
    Read words from a file and process them using spaCy's NER.
    
    Parameters:
    - file_path (str): Path to the input text file.
    - model: The spaCy NLP model.
    
    Returns:
    - recommended_locs (list): List of extracted locations.
    """
    i = 0
    recommended_locs = []
    with open(file_path, "r") as file:
        for line in file:
            word = line.strip()  # Remove leading/trailing whitespace
            if word:  # Skip empty lines
                i += 1
                # Process the word with spaCy
                doc = model(word)
                # Extract locations
                loc = get_locations_from_doc(doc)
                if loc:
                    recommended_locs.extend(loc)
                if i == 1000:
                    break
    return recommended_locs

def get_locations_from_doc(doc):
    """
    Extract locations from a spaCy Doc object.
    
    Parameters:
    - doc: A spaCy Doc object.
    
    Returns:
    - locations (list): List of location names identified in the text.
    """
    return [ent.text for ent in doc.ents if ent.label_ == "LOC"]
rec_loc_w = process_file("preprocessed_comments_words",model_t)


In [142]:
print(rec_loc_w)

[]
